In [3]:
import numpy as np
import pandas as pd
from datetime import datetime
from multiprocessing import Lock, Process, freeze_support

from sklearn.svm import SVR
from statsmodels.tools.eval_measures import rmse
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import os
import time

# objective function for PSO algorithm
# returns RMSE error for PSO particles
def SVRfx(particles, Xtrain, ytrain, Xtest, ytest):
    rmses = []
    for params in particles:
        model = make_pipeline(
            StandardScaler(),
            SVR(kernel='rbf', epsilon=params[0], gamma=params[1], C=params[2])
        )
        model.fit(Xtrain,ytrain)
        predictions = model.predict(Xtest)
        rmses.append(rmse(ytest,predictions))
    return rmses

PSOn = 7 # run PSO optimization of SVR parameters each 7 days, note: the smaller the number the more computationally expensive calculation
datasets = 'datasets' # folder name with datasets for individual periods in data folder
FILE_NAME = 'SVR_PSO.csv' # name of file with results

# PSO parameters
max_bound = np.array([1,1,1000]) # epsilon, gamma, C
min_bound = np.array([0.0001,0.0001,0.1])
bounds = (min_bound, max_bound) # lower and upper bounds for epsilon, gamma and C
options = {'c1': 0.5, 'c2': 0.5, 'w':0.2} # acceleration coefficients c1 and c2, inertia weight w

# division of hours into 8 groups for paralelization
hours_l = list(range(24))
hour_groups = np.array_split(hours_l, 8)

import pyswarms as ps
from sklearn.model_selection import train_test_split

write_lock = Lock()
# SVR calculation for given hours - each hourly period is forecasted by a separate SVR model trained 
# only on data from that period
def SVR_calc(hours):
    global PSOn, datasets, FILE_NAME
    global bounds, options

    for hour in hours:
        print(f'{str(hour).zfill(2)}:00')
        # load data from given period
        data = pd.read_csv(os.getcwd() + f'/data/SVR_dataset/{str(hour).zfill(2)}_00.csv',parse_dates=['datetime'],index_col='datetime').dropna()
        #Dropping ewma only for curve features purpose
        data.drop(['ewma'], axis = 1, inplace = True)

        # create list of dates to forecast
        dates = data.resample('D').count()[['grid1-loss']]
        dates = dates[dates['grid1-loss']>0]
        DATES = dates.loc['12-2019':].reset_index()['datetime'].apply(datetime.date).to_frame()['datetime']
        dates = dates.reset_index()['datetime'].apply(datetime.date).to_frame()

        i = 0
        for date in DATES:
            print(date)
            test_date = date
            date_index = dates.index[dates['datetime']==test_date][0]

            # create train set for PSO parameter optimization from 90% of all dates preceding the current test date
            # validation set is the last 10% of dates preceding the current test date
            X_valtrain, X_val, y_valtrain, y_val = train_test_split(data[:str(data.iloc[date_index-1].name.date())].iloc[:,1:],
                                                                    np.ravel(data[:str(data.iloc[date_index-1].name.date())].iloc[:,0]),
                                                                    test_size=.1, shuffle=False, stratify=None)

            # create train set from all dates preceding the current test date
            # test set is the 1 current test date
            X_train, X_test, y_train, y_test = train_test_split(data[:str(test_date)].iloc[:, 1:],
                                                                np.ravel(data[:str(test_date)].iloc[:, 0]),
                                                                test_size=1, shuffle=False, stratify=None)

            # if the current iteration is the PSOnth date, optimize SVR parameters with PSO
            if i % PSOn == 0:
                optimizer = ps.single.GlobalBestPSO(n_particles=30, dimensions=3, options=options, bounds=bounds)
                cost, pos = optimizer.optimize(SVRfx, iters=30, Xtrain=X_valtrain,ytrain=y_valtrain,Xtest=X_val,ytest=y_val,verbose=False)
            i = i + 1

            # create SVR model with feature normalization to (0,1) and forecast the next 15-minute period
            model = make_pipeline(
                StandardScaler(),
                SVR(kernel='rbf', epsilon=pos[0], gamma=pos[1], C=pos[2])
            )
            model.fit(X_train,y_train)
            predictions = model.predict(X_test)

            # create forecasts dataframe and append it to file with results
            forecasts = data[:str(test_date)].loc[str(test_date)][['grid1-loss']].reset_index() if hour > 0 else data[:str(test_date)].loc[[str(test_date)]][['grid1-loss']].reset_index()
            forecasts.rename({'grid1-loss':'real'},axis=1,inplace=True)
            forecasts['predicted'] = predictions

            # In your SVR_calc function, when writing to the file:
            with write_lock:
                forecasts.to_csv(f'{FILE_NAME}', mode='a', index=False, header=False)

if __name__ == "__main__":
    freeze_support() 
    
    # create empty file for results with header
    f = open(f'{FILE_NAME}', "w")
    f.write("datetime,real,predicted\n")
    f.close()
    
    processes = list()
    # run 8 paralel procecess 
    for hours in hour_groups:
        p = Process(target = SVR_calc, args = (hours,))
        processes.append(p)
        p.start()
    
    # Wait for all processes to finish
    for p in processes:
        p.join()

00:00
2019-12-01


KeyboardInterrupt: 